In [2]:
import random
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
spam = pd.read_csv(r"E:\KARTHIK\STUDY\NLP\SMSSpamCollection.txt", sep="\t", names=["labels","message"])

In [4]:
print(spam.head(3))

  labels                                            message
0    ham  Go until jurong point, crazy.. Available only ...
1    ham                      Ok lar... Joking wif u oni...
2   spam  Free entry in 2 a wkly comp to win FA Cup fina...


In [5]:
data_set=[]
for index,row in spam.iterrows():
    data_set.append((row["message"],row["labels"]))

In [6]:
print(data_set[:3])
print(len(data_set))

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'ham'), ('Ok lar... Joking wif u oni...', 'ham'), ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'spam')]
5572


In [7]:
stemmer=PorterStemmer()
lemmatizer=WordNetLemmatizer()

In [8]:
def preprocess(message, stem=True):
    document=message.lower()
    words=word_tokenize(document)
    stopword=stopwords.words("English")
    words=[word for word in words if word not in stopword]
    if stem:
        words=[stemmer.stem(word) for word in words]
    else:
        words=[lemmatizer.lemmatize(word, pos="v") for word in words]
    document=" ".join(words)
    return document

In [9]:
messages_set = []
for (message, label) in data_set:
    words_filtered = [e.lower() for e in preprocess(message, stem=False).split() if len(e) >= 3]
    messages_set.append((words_filtered, label))
print(len(messages_set))

5572


In [10]:
def get_words_in_message(messages):
    words_list=[]
    for message,label in messages:
        words_list.extend(message)
    return words_list

In [11]:
def featured_words(wordlist):
    aa=nltk.FreqDist(wordlist)
    features=aa.keys()
    return features

In [12]:
word_features=featured_words(get_words_in_message(messages_set))

In [13]:
print(len(word_features))

8393


In [14]:
slice_index=int(len(messages_set)*0.8)
print(slice_index)

4457


In [15]:
random.shuffle(messages_set)

In [16]:
train_messages, test_messages=messages_set[:slice_index], messages_set[slice_index:]
#print(len(train_messages))
#print(len(test_messages))

In [17]:
def extract_features(message):
    doc=set(message)
    features={}
    for word in word_features:
        features["contains (%s)" % word] = (word in doc) 
    return features

In [18]:
training_set = nltk.classify.apply_features(extract_features, train_messages)
test_set = nltk.classify.apply_features(extract_features, test_messages)

In [19]:
#print(training_set[:5])
print(len(training_set))
print(len(test_set))

4457
1115


In [20]:
spamclassifier = nltk.NaiveBayesClassifier.train(training_set)


In [21]:
print(nltk.classify.accuracy(spamclassifier, test_set))
#print(nltk.classify.accuracy(spamclassifier, training_set))


0.9838565022421525


In [22]:
## Testing a example message with our newly trained classifier
m = "CONGRATULATIONS!! As a valued account holder you have been selected to receive a £900 prize reward! Valid 12 hours only."
print('Classification result : ', spamclassifier.classify(extract_features(m.split())))

Classification result :  spam


In [23]:
m = "Hey, Whats up??"
print('Classification result : ', spamclassifier.classify(extract_features(m.split())))

Classification result :  ham
